In [1]:
from scipy import sparse
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score, mean_absolute_error

In [2]:
train_X = sparse.load_npz('../preprocessed_data/train_X.npz')
train_y = np.load('../preprocessed_data/train_y.npy')
test_X = sparse.load_npz('../preprocessed_data/test_X.npz')
test_y = np.load('../preprocessed_data/test_y.npy')


# Baseline


In [3]:
test_mean = [np.mean(test_y)] * len(test_y)
test_median = [np.median(test_y)] * len(test_y)


In [4]:
print('mean')

print(mean_absolute_error(test_y, test_mean))
print(mean_absolute_percentage_error(test_y, test_mean))
print(mean_squared_error(test_y, test_mean))
print(r2_score(test_y, test_mean))

print('median')

print(mean_absolute_error(test_y, test_median))
print(mean_absolute_percentage_error(test_y, test_median))
print(mean_squared_error(test_y, test_median))
print(r2_score(test_y, test_median))

mean
7.700773940473848
0.9286544865855756
107.6848077457088
0.0
median
7.382275644107646
0.7340606506803212
115.02961920995297
-0.06820657080605552


# model performance


In [5]:
svc = svm.LinearSVR()
svc.fit(train_X, train_y)
test_pred = svc.predict(test_X)
test_pred


array([ 6.86342519,  8.52360522,  3.59944704, ..., 10.56124303,
       11.02064937, 17.38139815])

In [6]:
print(mean_absolute_error(test_y, test_pred))
print(mean_absolute_percentage_error(test_y, test_pred))
print(mean_squared_error(test_y, test_pred))
print(r2_score(test_y, test_pred))



0.9159859113447364
0.08738091307197553
2.9886044150848052
0.9722467404859726


In [7]:
attribute_dic = pd.read_csv('../preprocessed_data/attribute_dic.csv', index_col=0)
attribute_dic['coef'] = np.abs(svc.coef_)
attribute_dic.sort_values(by=['coef'] , inplace=True, ascending=False)

print(attribute_dic.iloc[0,:])
print(attribute_dic.iloc[-1,:])

attribute_dic = attribute_dic.iloc[1:-1, :]
attribute_dic.to_csv('../preprocessed_data/svm_coef.csv')
attribute_dic

name        PULocationID
category               3
coef          415.552894
Name: 2, dtype: object
name        PULocationID
category             123
coef                 0.0
Name: 115, dtype: object


,name,category,coef
1,PULocationID,2,2.157783e+02
4,PULocationID,5,2.757300e+01
300,DOLocationID,49,1.472074e+01
457,DOLocationID,209,1.447828e+01
261,DOLocationID,10,1.300407e+01
...,...,...,...
172,PULocationID,182,1.174548e-03
117,PULocationID,125,2.220446e-16
207,PULocationID,219,1.110223e-16
258,DOLocationID,7,5.551115e-17


In [8]:
attribute_dic = attribute_dic.groupby('name').mean()
attribute_dic.sort_values(by=['coef'] , inplace=True, ascending=False)
attribute_dic.to_csv('../preprocessed_data/svm_coef_groupby.csv')
attribute_dic

,coef
name,
DOLocationID,2.626293
payment_type,2.492870
weekday,2.471128
PULocationID,1.847590
total_amount,0.310154
hour,0.263377
passenger_count,0.229287
tip_amount,0.186909
weather,0.153807
